# Training the OCR model :

**model selection** :
after searching a little bit i found some comparisons between some ocr models and easy ocr seemed to perform better than others espacially in license plates number detection ,and it is as well ,according to some random web page i found ,one of the easiest models to fine tune , so we will start with it and see how things go

**fine tunning easy ocr**
:
for this task i will be inspiring from a great [article](https://www.freecodecamp.org/news/how-to-fine-tune-easyocr-with-a-synthetic-dataset/) i found on freecodecamp on how to fine tune easy ocr

In [ ]:
#installing necessary librairies
!pip install fire
!pip install lmdb
!pip install opencv-python
!pip install natsort
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=c9649ce4f1b98f32f4c86450e17ff34e894768b6ee1c2156eef5ed49c618e948
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 11.1 MB/s eta 0:00:00


**note about the next cell:** you may  skip this cell and just use the  repository on our github.
In case you chose to clone the original ripository you will have to replace the original train.py,dataset.py and demo.py files with the modified ones in order to run the training and inference correctly  (you will find them in the rim_ai_competiton/deep-text-recognition-benchmark folder)

In [ ]:
#cloning the git repository  to help us run the fine tuning

#!git clone https://github.com/clovaai/deep-text-recognition-benchmark

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 499 (delta 0), reused 1 (delta 0), pack-reused 495 (from 1)
Receiving objects: 100% (499/499), 3.07 MiB | 13.11 MiB/s, done.
Resolving deltas: 100% (301/301), done.


In [ ]:
#preparing the labels for the model training
import pandas as pd
labels_csv=pd.read_csv("/content/drive/MyDrive/rim_ai_competition/train_labels.csv")#path to the train-labels.csv file provided in the competiton data
labels=open("/content/drive/MyDrive/deep-text-recognition-benchmark/labels.txt",'w')#creating a new labels.txt file to contain the labels
text=""
for i in range(204):

  text=text+f"{labels_csv.iloc[i]['img_id']}.jpg\t{labels_csv.iloc[i]['plate_number']} \n" #writing the labels in  the txt file

labels.write(text)
labels.close()
print(text)

img_1.jpg	8630AB06 
img_10.jpg	5115AM00 
img_100.jpg	7812AA02 
img_102.jpg	6421AA03 
img_103.jpg	0099AC08 
img_106.jpg	1277AY00 
img_107.jpg	5235AB07 
img_109.jpg	6602AB07 
img_111.jpg	3085AA12 
img_112.jpg	6563AR12 
img_115.jpg	6823AA12 
img_118.jpg	3664AB06 
img_119.jpg	5070AP00 
img_12.jpg	5008AR00 
img_120.jpg	3510AB07 
img_123.jpg	8519AA03 
img_126.jpg	1580AV00 
img_128.jpg	8900AA07 
img_129.jpg	1966AB08 
img_130.jpg	2534AB06 
img_133.jpg	3569AA04 
img_137.jpg	3079AM00 
img_139.jpg	1700AA05 
img_140.jpg	1417AX00 
img_142.jpg	5183AA12 
img_144.jpg	1709AA05 
img_146.jpg	9039AB08 
img_147.jpg	0153AB08 
img_148.jpg	1874AP00 
img_149.jpg	1428AL00 
img_150.jpg	9227AA08 
img_151.jpg	1579AA12 
img_156.jpg	6831AB08 
img_157.jpg	8594AA07 
img_158.jpg	0715AB07 
img_159.jpg	1973AA01 
img_16.jpg	1806AY00 
img_162.jpg	5220AA03 
img_163.jpg	7819AB08 
img_164.jpg	8146AB08 
img_165.jpg	6827AX00 
img_166.jpg	0368AS00 
img_17.jpg	3726AN00 
img_172.jpg	7526AB06 
img_175.jpg	1833AA12 
img_179.jpg	5778

In [ ]:
%cd deep-text-recognition-benchmark # moving to the deep text recognition benchmark

/content/deep-text-recognition-benchmark


In [ ]:
#getting the pretrained model from google drive (i got this pretrained model from he freecodecamp articule )
import gdown
file_id = "1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9"

# Construct the download URL
url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
output = "/content/TPS-ResNet-BiLSTM-Attn.pth"  # Change this to your desired filename and extension
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9
From (redirected): https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9&confirm=t&uuid=fd80254a-a268-417f-bbe5-3c068613f3ff
To: /content/TPS-ResNet-BiLSTM-Attn.pth
100%|██████████| 199M/199M [00:13<00:00, 15.0MB/s]


'/content/TPS-ResNet-BiLSTM-Attn.pth'

In [ ]:
#importing the lmdb module
import lmdb

In [ ]:
#transofming the dataset to lmdb format
!python create_lmdb_dataset.py \data /content/drive/MyDrive/rim_ai_competition/deep-text-recognition-benchmark/data/labels.txt \lmbd_output

204
Created dataset with 204 samples


**running the train.py file to train the model:**


*   train_data and val_data : path to the lmdb_output folder containing the cropped train images using yolo
*   saved_model : path to the pretrained easyocr model we downloaded form google drive
*   num_iterations : we trained the model for 300 iterations then for another 100 iterations so 400 iterations might be just enough


In [ ]:
!python train.py --train_data /content/drive/MyDrive/rim_ai_competition/deep-text-recognition-benchmark/lmbd_output --valid_data /content/drive/MyDrive/rim_ai_competition/deep-text-recognition-benchmark/lmbd_output  --select_data "/" --batch_ratio 1.0 --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn --batch_size 16 --data_filtering_off --workers 0 --batch_max_length 1000 --num_iter 400 --valInterval 5 --data_filtering_off --saved_model /content/TPS-ResNet-BiLSTM-Attn.pth

**running inference on the test data :**



*   saved_model:the path to the fine_tuned easy ocr model
*   image_folder : path to the folder containing the cropped test images using yolo


**note :** after running inference a log_demo_result.txt file will be created containing the predictions it should be in the deeptext recognition folder



In [ ]:
!python demo.py --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn --image_folder /content/drive/MyDrive/rim_ai_competition/croppet_test_number --saved_model /content/drive/MyDrive/rim_ai_competition/deep-text-recognition-benchmark/saved_models/best_model/best_norm_ED.pth

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from /content/drive/MyDrive/rim_ai_competition/deep-text-recognition-benchmark/saved_models/best_model/best_norm_ED.pth
/content/drive/MyDrive/rim_ai_competition/deep-text-recognition-benchmark/demo.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `